# EcoHome Energy Advisor - Agent Demo

This notebook demonstrates the EcoHome Energy Advisor agent with various real-world scenarios.

## Overview
The Energy Advisor helps homeowners:
- Optimize energy usage and reduce costs
- Maximize solar power utilization
- Get personalized recommendations based on weather and pricing
- Calculate potential savings from energy-efficient practices


## 1. Setup and Initialization


In [1]:
from agent import Agent
from datetime import datetime
from langchain_core.messages import AIMessage


In [2]:
# Define the system prompt for the Energy Advisor
ECOHOME_SYSTEM_PROMPT = """You are the EcoHome Energy Advisor, an AI assistant specialized in helping homeowners optimize their energy usage, reduce costs, and maximize the benefits of solar power.

Your primary goals are:
1. Help users minimize electricity costs by leveraging time-of-use pricing
2. Maximize the use of solar-generated electricity
3. Provide actionable recommendations for energy optimization
4. Calculate potential savings from energy-efficient practices

Available Tools:
- get_weather_forecast: Get weather forecasts including solar irradiance data
- get_electricity_prices: Get time-of-use electricity pricing by hour
- query_energy_usage: Query historical energy consumption data
- query_solar_generation: Query historical solar generation data
- get_recent_energy_summary: Get summary of recent energy usage and generation
- search_energy_tips: Search for energy-saving tips and best practices
- calculate_energy_savings: Calculate potential savings from optimizations. IMPORTANT: When users ask about savings from reducing usage (e.g., "if I reduce X by Y%, how much would I save?"), you MUST:
  1. First use query_energy_usage to get current usage data for the device/type
  2. Calculate the optimized usage based on the reduction percentage
  3. Then use calculate_energy_savings tool with the current usage, optimized usage, and device type
  4. Present the calculated savings to the user

Guidelines:
- Always use relevant tools to gather current data before making recommendations
- Consider weather forecasts when recommending solar-dependent activities
- Factor in electricity pricing (peak/off-peak) when suggesting optimal times
- Provide specific time recommendations when relevant
- Include cost calculations and savings estimates when possible
- Be clear, concise, and actionable in your responses
- If you need location information, ask the user or use the context provided
- Always explain your reasoning behind recommendations
- When users ask about potential savings from reducing energy usage, you MUST use the calculate_energy_savings tool to provide accurate calculations

When making recommendations:
- Prioritize using solar power when available
- Suggest off-peak hours for high-energy activities when solar isn't available
- Consider both cost savings and environmental impact
- Provide specific time windows for optimal actions

For savings calculations:
- If a user asks "how much would I save if I reduce [device] usage by X%", you MUST:
  1. Query current energy usage for that device type
  2. Calculate the reduced usage (current * (1 - X/100))
  3. Use calculate_energy_savings tool with these values
  4. Report the savings including monthly and annual projections
"""


In [3]:
# Initialize the agent
ecohome_agent = Agent(
    instructions=ECOHOME_SYSTEM_PROMPT,
)

print("✅ Energy Advisor agent initialized successfully!")
print(f"Available tools: {', '.join(ecohome_agent.get_agent_tools())}")


✅ Energy Advisor agent initialized successfully!
Available tools: get_weather_forecast, get_electricity_prices, query_energy_usage, query_solar_generation, get_recent_energy_summary, search_energy_tips, calculate_energy_savings


## 2. Helper Functions


In [4]:
def run_scenario(question, context=None, scenario_name=None):
    """
    Run a scenario with the Energy Advisor and display results.
    
    Args:
        question: The user's question
        context: Optional context (e.g., location)
        scenario_name: Optional name for the scenario
    """
    print("=" * 80)
    if scenario_name:
        print(f"SCENARIO: {scenario_name}")
    else:
        print("SCENARIO")
    print("=" * 80)
    print(f"\n📍 Context: {context if context else 'None'}")
    print(f"❓ Question: {question}\n")
    print("-" * 80)
    
    try:
        # Get response from agent
        response = ecohome_agent.invoke(question=question, context=context)
        
        # Extract and display the final response
        if isinstance(response, dict) and "messages" in response:
            messages = response["messages"]
            if messages:
                final_message = messages[-1]
                print("\n💡 Advisor Response:\n")
                print(final_message.content)
                
                # Show tools used
                print("\n" + "-" * 80)
                print("🔧 Tools Used:")
                tools_used = []
                for msg in messages:
                    if isinstance(msg, AIMessage) and hasattr(msg, 'tool_calls') and msg.tool_calls:
                        for tool_call in msg.tool_calls:
                            tool_name = tool_call.get("name", "unknown")
                            if tool_name not in tools_used:
                                tools_used.append(tool_name)
                
                if tools_used:
                    for tool in tools_used:
                        print(f"  • {tool}")
                else:
                    print("  (No tools were used)")
        else:
            print("\n💡 Advisor Response:\n")
            print(str(response))
            
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
    
    print("\n" + "=" * 80 + "\n")


## 3. Example Scenarios

Let's run the Energy Advisor through various real-world scenarios to see how it helps optimize energy usage.


### Scenario 1: EV Charging Optimization


In [5]:
run_scenario(
    question="When should I charge my electric car tomorrow to minimize cost and maximize solar power?",
    context="Location: San Francisco, CA",
    scenario_name="EV Charging Optimization"
)


SCENARIO: EV Charging Optimization

📍 Context: Location: San Francisco, CA
❓ Question: When should I charge my electric car tomorrow to minimize cost and maximize solar power?

--------------------------------------------------------------------------------

💡 Advisor Response:

To minimize costs and maximize solar power when charging your electric car tomorrow in San Francisco, here's the plan:

### Solar Generation and Electricity Pricing Overview
- **Solar Generation**: Solar power generation will start around 7 AM and peak around 11 AM to 1 PM. The solar irradiance will be highest during these hours.
- **Electricity Pricing**: 
  - **Off-Peak Rates**: 
    - 12 AM - 6 AM: $0.1191 to $0.1219
    - 10 PM - 12 AM: $0.1113 to $0.1157
  - **Peak Rates**: 
    - 6 AM - 10 PM: $0.1704 to $0.2419 (highest at 5 PM)

### Recommended Charging Times
1. **Charge During Solar Generation**: 
   - **Best Time**: 10 AM to 1 PM
   - **Reason**: This is when solar generation is at its peak, allowing 

### Scenario 2: Appliance Scheduling


In [6]:
run_scenario(
    question="When should I run my dishwasher and washing machine to save the most money?",
    context="Location: San Francisco, CA",
    scenario_name="Appliance Scheduling"
)


SCENARIO: Appliance Scheduling

📍 Context: Location: San Francisco, CA
❓ Question: When should I run my dishwasher and washing machine to save the most money?

--------------------------------------------------------------------------------

💡 Advisor Response:

To save the most money while running your dishwasher and washing machine, you should consider the following:

### Electricity Pricing
The electricity prices in San Francisco are structured as follows (in USD per kWh):

- **Off-Peak Hours** (Lowest Rates):
  - 12 AM - 6 AM: $0.1173 to $0.1297
  - 10 PM - 12 AM: $0.1112 to $0.1114

- **Peak Hours** (Highest Rates):
  - 6 AM - 10 AM: $0.1758 to $0.1894
  - 4 PM - 8 PM: $0.1726 to $0.2486

### Recommendations
1. **Run your dishwasher and washing machine during off-peak hours**:
   - **Best Time**: Between 10 PM and 6 AM, when rates are the lowest.
   - **Example**: Running them at 12 AM or 1 AM would cost you around $0.1173 to $0.1195 per kWh.

2. **Avoid peak hours**:
   - Do not 

### Scenario 3: Thermostat Optimization


In [7]:
run_scenario(
    question="What's the optimal thermostat setting for today to balance comfort and energy costs?",
    context="Location: San Francisco, CA",
    scenario_name="Thermostat Optimization"
)


SCENARIO: Thermostat Optimization

📍 Context: Location: San Francisco, CA
❓ Question: What's the optimal thermostat setting for today to balance comfort and energy costs?

--------------------------------------------------------------------------------

💡 Advisor Response:

Today's weather in San Francisco is expected to be cloudy with temperatures reaching a high of around 26.7°C (80°F) in the afternoon. Given this forecast, here are some recommendations for optimal thermostat settings to balance comfort and energy costs:

### Recommended Thermostat Settings:
- **Cooling Mode**: Set your thermostat to around **24-26°C (75-78°F)** during the day when temperatures peak. This range should keep your home comfortable without overworking your HVAC system.
- **Nighttime Setting**: In the evening, as temperatures drop, you can adjust the thermostat to around **21-23°C (70-74°F)** for comfort while sleeping.

### Energy Cost Considerations:
- **Peak Hours**: The highest temperatures and potent

### Scenario 4: Solar Power Maximization


In [8]:
run_scenario(
    question="How can I maximize my use of solar power tomorrow?",
    context="Location: San Francisco, CA",
    scenario_name="Solar Power Maximization"
)


SCENARIO: Solar Power Maximization

📍 Context: Location: San Francisco, CA
❓ Question: How can I maximize my use of solar power tomorrow?

--------------------------------------------------------------------------------

💡 Advisor Response:

To maximize your use of solar power tomorrow in San Francisco, here are some actionable recommendations based on the weather forecast and electricity pricing:

### Weather Forecast for Tomorrow
- **Morning (7 AM - 12 PM)**: Solar irradiance starts low but increases significantly by 8 AM, peaking around 10 AM with about 273.36 W/m².
- **Afternoon (1 PM - 4 PM)**: Solar irradiance remains high, peaking at 279.78 W/m² around 1 PM.
- **Evening (5 PM onwards)**: Solar generation drops significantly after 5 PM.

### Electricity Pricing
- **Peak Hours (6 AM - 7 PM)**: Higher rates ranging from $0.182 to $0.245 per kWh.
- **Off-Peak Hours (8 PM - 6 AM)**: Lower rates ranging from $0.1109 to $0.1298 per kWh.

### Recommendations
1. **Schedule Energy-Intensi

### Scenario 5: Energy Usage Analysis


In [9]:
run_scenario(
    question="How much did I spend on electricity in the last 24 hours and what was my biggest energy expense?",
    context="Location: San Francisco, CA",
    scenario_name="Energy Usage Analysis"
)


SCENARIO: Energy Usage Analysis

📍 Context: Location: San Francisco, CA
❓ Question: How much did I spend on electricity in the last 24 hours and what was my biggest energy expense?

--------------------------------------------------------------------------------

💡 Advisor Response:

In the last 24 hours, you spent a total of **$36.05** on electricity. 

Your biggest energy expense was from your **Electric Vehicle (EV)**, which consumed **206.45 kWh** and cost you **$26.46**. 

Here's a breakdown of your energy expenses:
- **EV**: $26.46
- **HVAC**: $5.20
- **Appliance**: $4.39

If you have any further questions or need assistance with optimizing your energy usage, feel free to ask!

--------------------------------------------------------------------------------
🔧 Tools Used:
  • get_recent_energy_summary




### Scenario 6: Energy-Saving Tips


In [10]:
run_scenario(
    question="What are the best ways to reduce my HVAC energy consumption?",
    context="Location: San Francisco, CA",
    scenario_name="Energy-Saving Tips"
)


SCENARIO: Energy-Saving Tips

📍 Context: Location: San Francisco, CA
❓ Question: What are the best ways to reduce my HVAC energy consumption?

--------------------------------------------------------------------------------


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



💡 Advisor Response:

Here are some effective ways to reduce your HVAC energy consumption:

1. **Smart Thermostats**: Install a smart thermostat that can automatically adjust temperatures when you're away. Setting your thermostat to 68°F in winter and 78°F in summer can save up to 10% on heating and cooling costs.

2. **Regular Maintenance**: Change air filters monthly to ensure better airflow and clean outdoor condenser coils annually. This helps your HVAC system operate more efficiently.

3. **Seal Air Leaks**: Use weatherstripping or caulk to seal air leaks around windows and doors. This prevents conditioned air from escaping and reduces the workload on your HVAC system.

4. **Insulation**: Add insulation to attics and walls to minimize heating and cooling needs. Proper insulation can significantly reduce energy consumption.

5. **Use Ceiling Fans**: Ceiling fans can help circulate air, making rooms feel cooler. This allows you to raise the thermostat temperature by about 4°F withou

### Scenario 7: Savings Calculation


In [11]:
run_scenario(
    question="If I reduce my air conditioning usage by 30%, how much money would I save per month?",
    context="Location: San Francisco, CA",
    scenario_name="Savings Calculation"
)


SCENARIO: Savings Calculation

📍 Context: Location: San Francisco, CA
❓ Question: If I reduce my air conditioning usage by 30%, how much money would I save per month?

--------------------------------------------------------------------------------

💡 Advisor Response:

If you reduce your air conditioning (HVAC) usage by 30%, you would save approximately **$32.24 per month**. 

### Breakdown:
- **Current Usage**: 613.68 kWh
- **Optimized Usage (after 30% reduction)**: 429.58 kWh
- **Savings**: 184.1 kWh
- **Cost per kWh**: $0.1751 (average price during peak hours)

### Annual Savings:
If you maintain this reduction, your annual savings would amount to approximately **$1176.36**.

This reduction not only helps you save money but also contributes to a more sustainable energy usage pattern. If you have any further questions or need more tips on optimizing your energy usage, feel free to ask!

--------------------------------------------------------------------------------
🔧 Tools Used:
  

### Scenario 8: Solar Generation Analysis


In [12]:
run_scenario(
    question="How much solar power did I generate in the last week and how does it compare to my usage?",
    context="Location: San Francisco, CA",
    scenario_name="Solar Generation Analysis"
)


SCENARIO: Solar Generation Analysis

📍 Context: Location: San Francisco, CA
❓ Question: How much solar power did I generate in the last week and how does it compare to my usage?

--------------------------------------------------------------------------------

💡 Advisor Response:

It appears that there is no recorded solar generation or energy usage data for the week of October 1 to October 8, 2023. This could be due to a variety of reasons, such as a lack of data collection or system issues.

If you have access to your solar generation and energy usage data through another source, you might want to check there. Alternatively, if you would like to look at a different date range or need assistance with anything else, please let me know!

--------------------------------------------------------------------------------
🔧 Tools Used:
  • query_solar_generation
  • query_energy_usage




### Scenario 9: Comprehensive Optimization Plan


In [13]:
run_scenario(
    question="Give me a comprehensive plan to optimize my energy usage for the next 3 days considering weather and pricing.",
    context="Location: San Francisco, CA",
    scenario_name="Comprehensive Optimization Plan"
)


SCENARIO: Comprehensive Optimization Plan

📍 Context: Location: San Francisco, CA
❓ Question: Give me a comprehensive plan to optimize my energy usage for the next 3 days considering weather and pricing.

--------------------------------------------------------------------------------

💡 Advisor Response:

Here's a comprehensive energy optimization plan for the next three days in San Francisco, CA, considering the weather forecast and time-of-use electricity pricing.

### Weather Forecast Overview
- **Day 1 (Today)**: Partly cloudy, temperatures ranging from 12.9°C to 22.5°C.
- **Day 2**: Similar conditions with temperatures from 12°C to 23°C.
- **Day 3**: Partly cloudy, temperatures from 11°C to 22°C.

### Electricity Pricing Overview
- **Off-Peak Rates**: 
  - 12 AM - 6 AM: $0.1224 - $0.1238
  - 10 PM - 12 AM: $0.1124 - $0.1205
- **Peak Rates**: 
  - 6 AM - 10 PM: $0.1702 - $0.2489

### Optimization Plan

#### **Day 1 (Today)**
- **Morning (6 AM - 10 AM)**: 
  - **Use Solar Power**: 

### Scenario 10: Quick EV Charging Question


In [14]:
run_scenario(
    question="I need to charge my EV from 20% to 80% battery. When is the cheapest time to do this today?",
    context="Location: San Francisco, CA",
    scenario_name="Quick EV Charging Question"
)


SCENARIO: Quick EV Charging Question

📍 Context: Location: San Francisco, CA
❓ Question: I need to charge my EV from 20% to 80% battery. When is the cheapest time to do this today?

--------------------------------------------------------------------------------

💡 Advisor Response:

To charge your EV from 20% to 80%, the cheapest times today are during the off-peak hours. Here are the best times to charge based on the electricity pricing:

- **From 10 PM to 12 AM**:
  - 10 PM: $0.1288 per kWh
  - 11 PM: $0.1139 per kWh

The lowest rate is at **11 PM** at **$0.1139 per kWh**. 

If you can wait until then, it would be the most cost-effective time to charge your EV.

--------------------------------------------------------------------------------
🔧 Tools Used:
  • get_electricity_prices




## 4. Summary

The Energy Advisor agent successfully demonstrates its ability to:
- ✅ Provide personalized recommendations based on weather and pricing data
- ✅ Optimize energy usage timing to minimize costs
- ✅ Maximize solar power utilization
- ✅ Calculate potential savings
- ✅ Offer actionable energy-saving tips
- ✅ Analyze historical energy data

The agent uses various tools including weather forecasts, electricity pricing, energy usage queries, and energy-saving tips to provide comprehensive and actionable advice.
